In [39]:
# -*- coding: UTF-8 -*-
"""PySteak Homework"""

# Import libraries
import csv
from pathlib import Path


# Set file paths for menu_data.csv and sales_data.csv
menu_filepath = Path('Resources/menu_data.csv')
sales_filepath = Path('Resources/sales_data.csv')


# Initialize list objects to hold our menu and sales data
menu = []
sales = []


# Read in the menu data into the menu list
with open(menu_filepath, 'r') as file:
    menu_csvreader = csv.reader(file, delimiter=',')
    
    # Skip the header and add to its own variable
    sales_header = next(menu_csvreader)

    # Loop through CSV and append to list
    for row in menu_csvreader:
        menu.append(row)

In [40]:
# Read in the sales data into the sales list
with open(sales_filepath, 'r') as file:
    sales_csvreader = csv.reader(file, delimiter=',')
    
    # Skip the header and add to its own variable
    sales_header = next(sales_csvreader)

    # Loop through CSV and append to list
    for row in sales_csvreader:
        sales.append(row)

In [57]:
# Initialize dict object to hold our key-value pairs of items and metrics
report = {}

# Initialize a row counter variable
row_count = 0

# Initialize a list to hold our month data for total months calculation
months_list = []

# Loop over every row in the sales list object
limiter = 0
for row in sales:
    # Line_Item_ID,Date,Credit_Card_Number,Quantity,Menu_Item
    # Initialize sales data variables
    quantity = int(row[3])
    sales_item = row[4]
    sales_date = row[1]

    # If the item value not in the report, add it as a new entry with initialized metrics
    # Naming convention allows the keys to be ordered in logical fashion, count, revenue, cost, profit
    if sales_item not in report:
        report[sales_item] = {
            "01-count": 0,
            "02-revenue": 0,
            "03-cogs": 0,
            "04-profit": 0,
            }  
    # Initialize variable to use to see if any of the sales items never match a menu item
    match_checker = False
    
    # For every row in our sales data, loop over the menu records to determine a match
    for row in menu:
        
        # Item,Category,Description,Price,Cost
        # Initialize menu data variables    
        menu_item = row[0]
        price = float(row[3])
        cost = float(row[4])
        profit = price - cost

        # Calculate profit of each item in the menu data
        # If the item value in our sales data is equal to the any of the items in the menu, then begin tracking metrics for that item
        if sales_item == menu_item:
            
            # Cumulatively add up the metrics for each item key
            report[sales_item]["01-count"] += quantity
            report[sales_item]["02-revenue"] += price * quantity
            report[sales_item]["03-cogs"] += cost * quantity
            report[sales_item]["04-profit"] += profit * quantity
            match_checker = True
    
    # Else, the sales item does not equal any fo the item in the menu data, therefore no match        
    # I think I only want to print no match if the loop never matches the sales item to a menu item
    # If I use if/else in the loop, it will print no match thousands of times
    # So I tried this method and it shows that all of our sales items do indeed find a match in the menu. 
    if match_checker == False:
        match_print(f"{sales_item} does not equal {menu_item}! NO MATCH!")

    # Pull out months
    sales_month = sales_date.split("-")[0]
    sales_year = sales_date.split("-")[2]
    sales_month_year = sales_month + "-" + sales_year
    # Create deduplicated list of months
    if sales_month_year not in months_list:
        months_list.append(sales_month_year)
    
    # Increment the row counter by 1
    row_count += 1


# Print total number of records in sales data
print (f"Rows in sales data = {row_count}\n")
print (f"Total number of months in the dataset = {len(months_list)}\n")

# Calculate most profitable and most costly items
highest_profit = 0
highest_profit_item = ""
highest_cost = 0
highest_cost_item = ""
for key, value in report.items():
    if value["04-profit"] > highest_profit:
        highest_profit = value["04-profit"]
        highest_profit_item = key
    if value["03-cogs"] > highest_cost:
        highest_cost = value["03-cogs"]
        highest_cost_item = key


# Write out report to a text file (won't appear on the command line output)
with open("output.txt", "w") as output_file:
    output_file.write("# Restaurant Analysis\n\n")
    output_file.write("## Summary of Data\n\n")
    output_file.write(f"Rows in sales data = {row_count}\n")
    output_file.write(f"Total number of months in the dataset = {len(months_list)}\n\n")
    output_file.write(f"The highest profit item was {highest_profit_item} with a total profit of ${highest_profit}. \n")
    output_file.write(f"The highest cost item was {highest_cost_item} with a total cost of ${highest_cost}.\n\n\n")
    output_file.write("## Sales Item Details\n\n")
    for key, value in report.items():
        output_file.write(key + " " + str(value) + "\n")

output_file.close()

Rows in sales data = 74124

Total number of months in the dataset = 24

